<h1>Посторение пайплайна</h1>

В качестве первой сети будем использовать cnn PyTorch'а

Сначала создадим кастомный датасет для нашей сети

In [189]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir  # Папка с изображениями
        self.labels = labels          # Список меток
        self.transform = transform    # Преобразования (если есть)

    def __len__(self):
        return len(self.labels)  # Возвращаем количество образцов

    def __getitem__(self, idx):
        img_name = self.labels[idx]['filename']  # Путь к изображению
        image = Image.open(img_name).convert('RGB')  # Открываем изображение
        label = self.labels[idx]['label']  # Получаем метку

        if self.transform:
            image = self.transform(image)  # Применяем преобразования

        return image, label  # Возвращаем изображение и метку

In [211]:
#получение списка всех папок(labels)
def list_dires(directory):
    all_items = []
    for root, dirs, files in os.walk(directory):
        for dird in dirs:
            all_items.append(dird)  # Добавляем директории
    return all_items
#получение списка всех изображений в папке
def list_files(directory):
    all_items = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            all_items.append(file)
    return all_items

items_prep = list_dires('preprocessed_images/train')
items_prep.sort()
classes=items_prep

In [215]:
def torch(**kwargs):
    image_dir = ''
    labels_train=[]
    for item in classes:
        for file in list_files('preprocessed_images/train/'+item):
            labels_train.append({'filename': 'preprocessed_images/train/'+item+'/'+file, 'label': int(item.replace('-',''))})
    # Создаем экземпляр датасета
    trainset = CustomDataset(image_dir=image_dir, labels=labels_train,transform=transform)

    # Создаем DataLoader
    trainloader = DataLoader(trainset, batch_size=4, shuffle=True)
    net = Net()
    criterion = nn.CrossEntropyLoss()  # Функция потерь для многоклассовой классификации
    optimizer = optim.SGD(net.parameters(), lr=kwargs.pop('lr'), momentum=kwargs.pop('momentum'))  # Стохастический градиентный спуск

    # Обучение модели
    for epoch in range(kwargs.pop('epoch')):  # Проходим по всему набору данных 2 раза
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labeld = data
            optimizer.zero_grad()  # Обнуляем градиенты
            outputs = net(inputs)  # Прямой проход
            loss = criterion(outputs, labeld)  # Вычисляем потерю
            loss.backward()  # Обратный проход
            optimizer.step()  # Обновляем параметры
            running_loss += loss.item()
            if i % 2000 == 1999:  # Печатаем каждые 2000 мини-батчей
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    labels_test=[]
    for item in classes:
        for file in list_files('preprocessed_images/test/'+item):
            labels_test.append({'filename': file, 'label': int(item.replace('-',''))})

    testset = CustomDataset(image_dir=image_dir, labels=labels_test,transform=transform)

    testloader = DataLoader(testset, batch_size=4, shuffle=True)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 1000 test images: {100 * correct // total} %')

Итоговый метод пайплайна (увы, меньше чем ожидалось)

In [216]:
def pipeline(model, **kwargs):
    if model == 'torch':
        torch(**kwargs)

In [217]:
pipeline('torch', lr = 0.001, momentum = 0.9, epoch =2)

KeyboardInterrupt: 